In [1]:
import numpy as np
from ROAR.utilities_module.occupancy_map import OccupancyGridMap
from pathlib import Path
import logging
from pathlib import Path
from ROAR_Sim.configurations.configuration import Configuration as CarlaConfig
from ROAR_Sim.carla_client.carla_runner import CarlaRunner
from ROAR.agent_module.pure_pursuit_agent import PurePursuitAgent
from ROAR.configurations.configuration import Configuration as AgentConfig
from ROAR.agent_module.michael_pid_agent import PIDAgent
from ROAR.agent_module.special_agents.waypoint_generating_agent import WaypointGeneratigAgent
from ROAR.agent_module.special_agents.recording_agent import RecordingAgent
from ROAR.agent_module.rl_e2e_ppo_agent import RLe2ePPOAgent
import argparse
from ROAR.utilities_module.data_structures_models import Transform, Location
import cv2
from ROAR_gym.configurations.ppo_configuration import misc_params, spawn_params

pygame 2.1.2 (SDL 2.0.18, Python 3.8.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
agent_config = AgentConfig.parse_file(Path("./ROAR_gym/configurations/agent_configuration.json"))
agent = RLe2ePPOAgent(vehicle=None, agent_settings=agent_config)

337665 419


In [4]:
print(len(agent.plan_lst))

337246


9.147531620648358e-07 0.07622707477629088 337665


list

In [6]:
new_waypoints = agent.plan_lst
remove_index = []
for i in range(1, len(new_waypoints)):
        pt1 = new_waypoints[i-1]
        pt2 = new_waypoints[i]
        dist = np.linalg.norm (pt1 - pt2)
        if dist <= 1e-3:
            remove_index.append(i)

In [7]:
print(len(agent.plan_lst), len(remove_index))

337665 419


In [8]:
for i in range(len(remove_index)):
    agent.plan_lst.pop(remove_index[i] - i)

In [9]:
print(len(agent.plan_lst), len(remove_index))

337246 419


In [10]:
dist_lst = np.array(agent.mission_planner.waypoints_list_check(agent.plan_lst))

0.0010011512503163556 0.07622707477629088 337246


In [7]:
#file = open("C://Users//roar//Desktop//ROAR_record//ROAR_record//data//output//border_waypoints//right_side_major_map_waypoints.txt")
file = open(".//data//waypoints//major_map_waypoints2.txt")
full_list = []
for r in file:
    cur_list = []
    seg = r.rstrip().split(',')
    for item in seg:
        cur_list.append(float(item))
    full_list.append(cur_list)
waypoints = np.array(full_list)
#waypoints[:, 3:] = [0, 0, 0]
#waypoints[:, :3] += np.array([50, -8, 295])
waypoints[0]

array([2599.19995117,  104.74610138, 4372.60009766,    0.        ,
          0.        ,   10.00061798])

In [8]:
len(waypoints)
print(waypoints[0])

[2599.19995117  104.74610138 4372.60009766    0.            0.
   10.00061798]


In [3]:
agent_config = AgentConfig.parse_file(Path("./ROAR_gym/configurations/agent_configuration.json"))
agent = RLe2ePPOAgent(vehicle=None, agent_settings=agent_config)

TypeError: interpolate_waypoints() missing 1 required positional argument: 'thres'

In [9]:
agent_config = AgentConfig.parse_file(Path("./ROAR/configurations/carla/carla_agent_configuration.json"))
agent = RLe2ePPOAgent(vehicle=None, agent_settings=agent_config)

loadmap = OccupancyGridMap(agent)
#loadmap.load_from_file(Path("C://Users//roar//Desktop//ROAR_record//ROAR_record//data//output//occupancy_map//occu_map.npy"))
loadmap.load_from_file(Path("data//cleaned_occ_map//berkeley_major_cleaned_occu_map.npy"))

In [10]:
local_transform = Transform.from_array(waypoints[0])
#local_map = loadmap.get_map(transform = local_transform)
local_map = loadmap.get_map(transform = local_transform, view_size=(100, 100), boundary_size=(20, 20), show_agent= True)
cv2.imshow("local_map", local_map) # uncomment to show occu map
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
count = 0
k = 10
for i in range(len(waypoints)//k):
    #count += 1
    #print(waypoints[k*i])
    loadmap.update_to_one(np.array([waypoints[i*k][:3]]))
    transform = Transform.from_array(waypoints[k*i])
    local_map = loadmap.get_map(transform = transform)
    # x, y = points[0], points[2]
    cv2.imshow("local_map", local_map) # uncomment to show occu map
    cv2.waitKey(1)
    #cv2.destroyAllWindows()
cv2.destroyAllWindows()
print(count)

KeyboardInterrupt: 

# Getting spawn index for reward line

In [2]:
spawn_position = spawn_params['spawn_position']
file = open("./ROAR_gym/tempout.txt")
reward_lines = []
for r in file:
    seg = r.rstrip().split(' ')
    reward_lines.append([float(seg[0]), float(seg[1])])
reward_lines = np.array(reward_lines)
print(reward_lines.shape)

(3373, 2)


In [3]:
closest_index_dict = {}
for id in spawn_position:
    pt = spawn_position[id]
    closest_index = np.argmin(np.linalg.norm(reward_lines - np.array([pt[0], pt[2]]), axis = 1))
    closest_index_dict[id] = closest_index + 1
print(closest_index_dict)

{0: 70, 1: 317, 2: 467, 3: 727, 4: 993, 5: 1269, 6: 1512, 7: 1906, 8: 2024, 9: 2298, 10: 2923, 11: 3179, 12: 4}


In [6]:
print(closest_index_dict.values())

dict_values([70, 317, 467, 727, 993, 1269, 1512, 1906, 2024, 2298, 2923, 3179, 4])


In [7]:
reward_lines[317]

array([1345.54311752, 4259.20436535])